In [2]:
import scipy.stats
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import re


In [3]:
import re
# Function to parse the string and remove unwanted characters
def parse_string(text_string):
    """"
        Parse the input string and tokenize it using regular expressisons:
        First clean the string such that it does not have any punctuation or number, it must only have a-z and A-Z.
        Please note that while doing this, the spaces much not get disturbed, but in case of multiple spaces convert 
        them to one space.
        Then convert the string to lower case and return its words as a list of strings.
        
        Example:
        --------
        Input :  computer scien_tist-s are,,,  the  rock__stars of tomorrow_ <cool>  ????
        Output:  ['computer', 'scientists', 'are', 'the', 'rockstars', 'of', 'tomorrow']
        
        Parameters:
        ----------
        string: string to be parsed...
        re: regular expression to be used for the tokenization.
        
        Returns:
        ---------
        list of tokens extracted from the string...
    """ 
    # Check if the input is bytes, and decode it to string if necessary
    if isinstance(text_string, bytes):
        text_string = text_string.decode('utf-8')
    regex_pat = r'[^\w\s]'
    # Remove HTML tags and text inside parentheses
    new_string = re.sub(r'<[^>]*>|\([^)]*\)', '', text_string)
    # Remove non-alphanumeric characters
    new_string = re.sub(regex_pat, '', new_string)
    regex_pat2 = r'-+'
    # Replace multiple hyphens with a single space
    new_string = re.sub(regex_pat2, ' ', new_string)
    return new_string


In [4]:

def parse_string1(text_string): 
    """"
        Parse the input string and tokenize it using regular expressisons:
        First clean the string such that it does not have any punctuation or number, it must only have a-z and A-Z.
        Please note that while doing this, the spaces much not get disturbed, but in case of multiple spaces convert 
        them to one space.
        Then convert the string to lower case and return its words as a list of strings.
        
        Example:
        --------
        Input :  computer scien_tist-s are,,,  the  rock__stars of tomorrow_ <cool>  ????
        Output:  ['computer', 'scientists', 'are', 'the', 'rockstars', 'of', 'tomorrow']
        
        Parameters:
        ----------
        string: string to be parsed...
        re: regular expression to be used for the tokenization.
        
        Returns:
        ---------
        list of tokens extracted from the string...
    """

    # Check if the input is bytes, and decode it to string if necessary
    if isinstance(text_string, bytes):
        text_string = text_string.decode('utf-8')
    regex_pat = r'[^\w\s]'
    new_string = re.sub(r'<[^>]*>|\([^)]*\)', '', text_string)
    new_string = re.sub(regex_pat,'',new_string)
    regex_pat2 = r'-+'
    new_string = re.sub(regex_pat2,'',new_string)
    # new_string = re.sub(r'<[^>]*>|\([^)]*\)', '', new_string)
    word_tokens = re.findall(r'\w+', new_string)
    reg_pat3 = r'[^a-zA-Z]'
    new_clean = []
    for word in word_tokens:
        temp = re.sub(reg_pat3, '', word)
        new_clean.append(temp)
    return new_clean

clean = parse_string1('''        computer scien_tist-s are,,,  the 
                    # rock__stars of tomorrow_ <cool>  ????''')
clean

['computer', 'scientists', 'are', 'the', 'rockstars', 'of', 'tomorrow']

In [6]:
def parse_file(filename): # Parse a given file
    """"
        Parameters:
        ----------
        filename: name of text file to be read
   
        
        Returns:
        ---------
        read file as raw string (with \n, \t, \r, etc included)
    """
    lines = []
    with open(filename, 'r') as file:
        lines = file.read()
        #for line in file:
        #    lines.append(line)
    return lines
lines = parse_file('./neg/cv000_29416.txt')
lines

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience membe

In [7]:
def files_to_strings(X):   
    """
        Read an array (or list) of files where each file content is read in a string...
        Input:
        -------
        X an array (or list) of file names
        
        Returns:
        --------
        X as a numpy array with each row containing a read string from the file...
    """
    all_file_data = []
    for file in X:
        current_file_data = []
        with open(file, 'r') as file_content:
            current_file_data = file_content.read() 
        all_file_data.append(current_file_data)
    return np.array(all_file_data)

In [9]:
X = np.array(["./neg/cv000_29416.txt", "./pos/cv000_29590.txt"])
content = files_to_strings(X)

In [10]:
content[0]

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience membe

In [11]:
if content[0]==lines:
    print('Same')

Same


In [12]:
file2 = parse_file("./pos/cv000_29590.txt")
if content[1] == file2:
    print('Same')

Same


In [13]:
# Removing stopwords
def remove_stopwords(tokens):
    new_tokens=[]
    for t in tokens:
        if t not in stop_words and t!='':
            new_tokens.append(t)
            
    return new_tokens

In [15]:
# let's read the stopwords wali file and then make an array of stopwords
file1 = './english.stop'
stop_words = []
with open(file1,'r') as fil:
    for words in fil:
        stop_words.append(words)
stop_words = [word.strip() for word in stop_words]

#### Test Case for the above code

In [17]:
from nose.tools import assert_equal, assert_list_equal
import numpy as np
assert_list_equal(parse_string1("computer scien_tist-s are,,,  the  rock__stars of tomorrow_ <cool>  ????"),
        [u'computer', u'scientists', u'are', u'the', u'rockstars', u'of', u'tomorrow'], "Incorrect cleanning")


strings = files_to_strings(np.array(["./neg/cv000_29416.txt", "./pos/cv000_29590.txt"]))
with open("./neg/cv000_29416.txt") as f:
    text = f.read()

assert_equal(strings[0], text, "At first index should be text of first file")
assert_equal(strings.shape, (2,), "Shape must be (2,) for two files in list")

#### Preparing Test and Training Data

In [18]:
import pandas as pd
import tools as t

tools.py is being imported


In [19]:
import os
posfiles = [os.path.join('./pos', file) for file in os.listdir('./pos')]
negfiles = [os.path.join('./neg', file) for file in os.listdir('./neg')]

In [20]:
import numpy as np
plabels=['pos']*(len(posfiles))
nlabels=['neg']*(len(posfiles))
labels=np.concatenate((plabels,nlabels)) # concatenate the +ve and -ve labels
tX=np.concatenate((posfiles,negfiles))
print ("Training data Dimensions =", tX.shape," Training labels dimensions=", labels.shape)

Training data Dimensions = (2000,)  Training labels dimensions= (2000,)


In [21]:
X=files_to_strings(tX) # read files and convert each file into set of strings and return an numpy array
X = X.reshape((X.shape[0], 1))
#Split the data into two halves training and test set...
#traindata,trainlabels,testdata,testlabels=t.split_data(X,labels)
indices = np.random.permutation(len(X))
split_point = int(len(X) * 0.8)  # 80% training, 20% test
train_idx, test_idx = indices[:split_point], indices[split_point:]
traindata, testdata = X[train_idx], X[test_idx]
trainlabels, testlabels = labels[train_idx], labels[test_idx]
#Find the classes to train
classes=np.unique(labels)

In [22]:
text = traindata[0][0]

#### Before getting into Naive Bayes we will tokenize and preprocess the data

In [23]:
# Working on training data
train_data_tokens = []
for content in traindata:
# for file in traindata:
    # for this current file extract iska content
    #content = parse_file(file)
    # now that we have the content 
    # and as content is array of lines
    # for each line we will extract tokens 
    tokens = []
    for line in content:
        temp_tokens = parse_string(line)
        for ts in temp_tokens:
            tokens.append(ts)
    new_tokens = remove_stopwords(tokens)
    # file 1 k new tokens are on first index
    for t in new_tokens:
        train_data_tokens.append(t)

In [24]:
len(train_data_tokens)

1187898